In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
results = np.load("../src/evaluation/results.npz", allow_pickle=True)["results"].item()

In [ ]:
# def make_table()
database_name = "COCO"
model_names = ["vgg16", "resnet50"]
methods = ["0", "0.5", "1", "perfect", "grad_cam", "rise",  "extremal_perturbations",  "igos_pp", "rt_saliency", "explainer"]
methods_names = ["0", "0.5", "1", "Ideal", "GradCam", "RISE",  "Fong 19",  "iGOS++", "Dabkowski 17",  "Explainer", ]
metrics = ["d_f1", "c_f1", "acc", "d_IOUs", "c_IOU" ,"aucs",'sal', 'over', 'background_c', 'mask_c']
metrics_names = ["Mean F1", "Cont F1", "Accuracy", "Discr. IOU", "Cont. IOU", "AUC", 'Saliency', 'Accuracy', 'Background cov.', 'Object cov.']

indexes = pd.MultiIndex.from_product([metrics_names, model_names], names=["Metric", "Network"])

m = np.zeros([len(metrics)*2, len(methods)])
for i,metric in enumerate(metrics):
    for j, method in enumerate(methods):
        for k in [0,1]:
            try:
                v = results[database_name][model_names[k]][method][metric]
                m[2*i+k,j] = "{:.2f}".format(np.mean(np.array([e for e in v if e is not None])))
            except:
                m[2*i+k,j] = np.nan
df = pd.DataFrame(m, index=indexes, columns=methods_names)


print(df.to_latex(index=True, escape=False))


In [ ]:
model_name = "vgg16"
plt.figure(figsize=(6, 4))
x = np.arange(0.1, 1, .1)
for method, method_name in zip(methods, methods_names):
    if method_name in ["0.5", "Ideal"]: continue
    if method_name in ["0", "1"]:
        plt.plot(x, np.mean(np.array(results[database_name][model_name][method]["a_f1s"]), axis=0), '--', label=method_name)
    else:
        plt.plot(x, np.mean(np.array(results[database_name][model_name][method]["a_f1s"]), axis=0), label=method_name)
plt.legend(bbox_to_anchor=(-0.03, 1.0, 1, 0.2), loc="lower left", ncol=4)
plt.xlabel("Threshold")
plt.ylabel("F1 score")
plt.savefig("f1s_{}_{}.pdf".format(database_name, model_name))